In [1]:
# Loading Libraries, Remember to run this cell!
library(tidyverse)
library(repr)
library(tidymodels)

options(repr.matrix.max.rows = 6)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘tidymodels’ was built under R version 4.0.2”
── Attaching packages ────────────────────────────────────── tidymodels 0.1.1 ──

✔ broom     0.7.0      ✔ recipes   0.1.13
✔ dials     0.0.9      ✔ rsample   0.0.7 
✔ infer     0.5.4      ✔ tune      0.1.1 
✔ modeldata 0.0.2      ✔ workflows 0.2.0 
✔ parsnip   0.1.3    

In [2]:
# Loading in the data
# Data read from https://drive.google.com/uc?export=download&id=1_MECmUXZuuILYeEOfonSGqodW6qVdhsS
player_stats <- read_csv("player_stats.csv")
colnames(player_stats) <- make.names(colnames(player_stats))
#   Remove unnecessary columns
player_stats_clean <- player_stats %>%
    select(Age:Peak.Elo.Rating, Retired, -Country, -Wikipedia, -Current.Rank,-Name, -Seasons, -Prize.Money, -Active, -Favorite.Surface, -Best.Elo.Rank) %>%
#   Mutating columns with as_factor()
    mutate(Dominate.Hand = as_factor(Plays)) %>%
    select(-Plays) %>%
    mutate(Backhand = as_factor(Backhand)) %>%
#   Cleaning up columns with string values
    mutate(Height = strtoi(str_remove(Height, " cm"))) %>%
    mutate(Year.Experience = 2020 - Turned.Pro) %>%
    mutate(Age = strtoi(substr(Age, 0, 2))) %>%
    mutate(Current.Elo.Rank = strtoi(substr(Current.Elo.Rank, 0, nchar(Current.Elo.Rank)-7))) %>%
    filter(!is.na(Turned.Pro)) %>%
    filter(is.na(Retired))
player_stats
player_stats_clean

Warning message:
“Missing column names filled in: 'X1' [1]”
Parsed with column specification:
cols(
  .default = col_character(),
  X1 = col_double(),
  `Turned Pro` = col_double(),
  Seasons = col_double(),
  Titles = col_double(),
  `Best Season` = col_double(),
  Retired = col_double(),
  Masters = col_double(),
  `Grand Slams` = col_double(),
  `Davis Cups` = col_double(),
  `Team Cups` = col_double(),
  Olympics = col_double(),
  `Weeks at No. 1` = col_double(),
  `Tour Finals` = col_double()
)

See spec(...) for full column specifications.



X1,Age,Country,Plays,Wikipedia,Current.Rank,Best.Rank,Name,Backhand,Prize.Money,⋯,Facebook,Twitter,Nicknames,Grand.Slams,Davis.Cups,Web.Site,Team.Cups,Olympics,Weeks.at.No..1,Tour.Finals
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
0,26 (25-04-1993),Brazil,Right-handed,Wikipedia,378 (97),363 (04-11-2019),Oscar Jose Gutierrez,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1,18 (22-12-2001),United Kingdom,Left-handed,Wikipedia,326 (119),316 (14-10-2019),Jack Draper,Two-handed,"$59,040",⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,32 (03-11-1987),Slovakia,Right-handed,Wikipedia,178 (280),44 (14-01-2013),Lukas Lacko,Two-handed,"US$3,261,567",⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
497,23 (14-03-1996),Netherlands,Left-handed,Wikipedia,495 (60),342 (05-08-2019),Gijs Brouwer,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
498,24 (17-05-1995),Ukraine,NA,Wikipedia,419 (81),419 (20-01-2020),Vladyslav Orlov,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
499,22 (26-03-1997),Tunisia,Left-handed,Wikipedia,451 (69),408 (24-12-2018),Aziz Dougaz,Two-handed,"$61,984",⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


Age,Best.Rank,Backhand,Height,Turned.Pro,Current.Elo.Rank,Peak.Elo.Rating,Retired,Dominate.Hand,Year.Experience
<int>,<chr>,<fct>,<int>,<dbl>,<int>,<chr>,<dbl>,<fct>,<dbl>
32,44 (14-01-2013),Two-handed,185,2005,144,1886 (06-02-2012),NA,Right-handed,15
27,17 (11-01-2016),Two-handed,193,2008,100,2037 (01-02-2016),NA,Right-handed,12
22,31 (20-01-2020),Two-handed,NA,2015,33,1983 (20-01-2020),NA,Right-handed,5
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
28,74 (19-02-2018),Two-handed,NA,2008,143,1904 (12-02-2018),NA,Right-handed,12
26,249 (24-12-2018),Two-handed,NA,2009,180,1679 (10-01-2020),NA,Right-handed,11
26,4 (06-11-2017),One-handed,185,2011,6,2211 (18-11-2019),NA,Right-handed,9


In [10]:
# Making the training and testing splits for each of the predictions
backhand_split <- initial_split(player_stats_clean, prop = .6, strata = Backhand)
backhand_training <- training(backhand_split)
backhand_testing <- testing(backhand_split)

hand_split <- initial_split(player_stats_clean, prop = .6, strata = Dominate.Hand)
hand_training <- training(hand_split)
hand_testing <- testing(hand_split)

backhand_training
backhand_testing
hand_training
hand_testing

Age,Best.Rank,Backhand,Height,Turned.Pro,Current.Elo.Rank,Peak.Elo.Rating,Retired,Dominate.Hand,Year.Experience
<int>,<chr>,<fct>,<int>,<dbl>,<int>,<chr>,<dbl>,<fct>,<dbl>
27,17 (11-01-2016),Two-handed,193,2008,100,2037 (01-02-2016),NA,Right-handed,12
22,31 (20-01-2020),Two-handed,NA,2015,33,1983 (20-01-2020),NA,Right-handed,5
19,17 (14-10-2019),Two-handed,NA,2017,51,1992 (12-08-2019),NA,Right-handed,3
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
24,27 (10-06-2019),Two-handed,NA,2013,95,1913 (10-06-2019),NA,Right-handed,7
28,74 (19-02-2018),Two-handed,NA,2008,143,1904 (12-02-2018),NA,Right-handed,12
26,249 (24-12-2018),Two-handed,NA,2009,180,1679 (10-01-2020),NA,Right-handed,11


Age,Best.Rank,Backhand,Height,Turned.Pro,Current.Elo.Rank,Peak.Elo.Rating,Retired,Dominate.Hand,Year.Experience
<int>,<chr>,<fct>,<int>,<dbl>,<int>,<chr>,<dbl>,<fct>,<dbl>
32,44 (14-01-2013),Two-handed,185,2005,144,1886 (06-02-2012),NA,Right-handed,15
23,4 (09-09-2019),Two-handed,NA,2014,5,2243 (14-10-2019),NA,Right-handed,6
28,134 (18-11-2019),NA,NA,2008,NA,NA,NA,Right-handed,12
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
32,84 (28-09-2015),Two-handed,NA,2006,NA,1859 (13-02-2018),NA,Left-handed,14
24,92 (19-08-2019),Two-handed,NA,2016,186,1733 (18-02-2019),NA,Right-handed,4
26,4 (06-11-2017),One-handed,185,2011,6,2211 (18-11-2019),NA,Right-handed,9


Age,Best.Rank,Backhand,Height,Turned.Pro,Current.Elo.Rank,Peak.Elo.Rating,Retired,Dominate.Hand,Year.Experience
<int>,<chr>,<fct>,<int>,<dbl>,<int>,<chr>,<dbl>,<fct>,<dbl>
32,44 (14-01-2013),Two-handed,185,2005,144,1886 (06-02-2012),NA,Right-handed,15
27,17 (11-01-2016),Two-handed,193,2008,100,2037 (01-02-2016),NA,Right-handed,12
22,31 (20-01-2020),Two-handed,NA,2015,33,1983 (20-01-2020),NA,Right-handed,5
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
28,74 (19-02-2018),Two-handed,NA,2008,143,1904 (12-02-2018),NA,Right-handed,12
26,249 (24-12-2018),Two-handed,NA,2009,180,1679 (10-01-2020),NA,Right-handed,11
26,4 (06-11-2017),One-handed,185,2011,6,2211 (18-11-2019),NA,Right-handed,9


Age,Best.Rank,Backhand,Height,Turned.Pro,Current.Elo.Rank,Peak.Elo.Rating,Retired,Dominate.Hand,Year.Experience
<int>,<chr>,<fct>,<int>,<dbl>,<int>,<chr>,<dbl>,<fct>,<dbl>
19,17 (14-10-2019),Two-handed,NA,2017,51,1992 (12-08-2019),NA,Right-handed,3
23,4 (09-09-2019),Two-handed,NA,2014,5,2243 (14-10-2019),NA,Right-handed,6
22,25 (05-08-2019),Two-handed,NA,2015,56,1999 (05-08-2019),NA,Right-handed,5
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
35,6 (05-01-2009),Two-handed,183,2002,63,2124 (02-02-2009),NA,Right-handed,18
29,3 (21-11-2016),Two-handed,196,2008,21,2269 (11-07-2016),NA,Right-handed,12
29,189 (13-07-2015),Two-handed,NA,2013,NA,NA,NA,Right-handed,7


In [11]:
set.seed(1234)
# Creating the Recipes
backhand_recipe <- recipe(Backhand ~ Current.Elo.Rank + Year.Experience, data = backhand_training) %>%
    step_scale(all_predictors()) %>%
    step_center(all_predictors())

hand_recipe <- recipe(Dominate.Hand ~ Current.Elo.Rank + Year.Experience, data = hand_training) %>%
    step_scale(all_predictors()) %>%
    step_center(all_predictors())

backhand_recipe
hand_recipe

Data Recipe

Inputs:

      role #variables
   outcome          1
 predictor          2

Operations:

Scaling for all_predictors()
Centering for all_predictors()

Data Recipe

Inputs:

      role #variables
   outcome          1
 predictor          2

Operations:

Scaling for all_predictors()
Centering for all_predictors()

In [16]:
set.seed(1234)

# Creating the vfolds
backhand_vfold <- vfold_cv(backhand_training, v = 2, strata = Backhand)
hand_vfold <- vfold_cv(hand_training, v = 2, strata = Dominate.Hand)

In [17]:
set.seed(1234)
# Making the tune model to choose the best value of k
knn_tune <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) %>%
        set_engine("kknn") %>%
        set_mode("classification")

In [18]:
set.seed(1234)
# Collecting metrics except its broken
backhand_results <- workflow() %>%
    add_recipe(backhand_recipe) %>%
    add_model(knn_tune) %>%
    tune_grid(resamples = backhand_vfold, grid = 10)
hand_results <- workflow() %>%
    add_recipe(hand_recipe) %>%
    add_model(knn_tune) %>%
    tune_grid(resamples = hand_vfold, grid = 10)

backhand_results
hand_results

x Fold1: model 1/1 (predictions): Error: Problem with `mutate()` input `.row`.
[...

x Fold2: model 1/1 (predictions): Error: Problem with `mutate()` input `.row`.
[...

Warning message:
“All models failed in tune_grid(). See the `.notes` column.”
x Fold1: model 1/1 (predictions): Error: Problem with `mutate()` input `.row`.
[...

x Fold2: model 1/1 (predictions): Error: Problem with `mutate()` input `.row`.
[...

Warning message:
“All models failed in tune_grid(). See the `.notes` column.”


splits                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

splits                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          